In [ ]:
#--------------------------------------------------------------------------------
#
### IO TEST ORCHESTRATOR ###
#
# This is an IPython Notebook responsible for experiments for IO and disk operations testing.
# It is intended to be executed unattended for testing purposes.
#
# test_orchestrator should be used as is, simply running it as IPython Notebook, 
# or copied-and-pasted into a plain python script and called from bash.
#
# test_orchestrator does not need any input parameter as experiments parameters 
# are specified in the code itself. Please refer to the "Workload parameters" section.
#
#
# REQUIREMENTS:
# -------------
# This script relies on external test classes to run the experiments. 
# All the intelligence (e.g., workload generation, writing policies, logging, sanity checks, 
# error handling capabilities) are demanded to the classes here imported and used to run the tests.
#
# Given the nature of the experiments, it is required to have write permissions to disk.
# It is possible to specify the output directory using the "OUTPUT_FOLDER" parameter.
#
#
#
# FUNCTIONING:
# -----------
# By now, three classes of tests are available:
#  1. PlainWrite (from file test_write)  -- opens a file for write, writes all the content, and closes the file.
#  2. Flush      (from file test_flush)  -- opens a file for write, flushes a specified amount of bytes, waits a
#                                           specified amount of time, and closes the file.
#                                           flush and wait are repeated until there are bytes to be written.
#  3. Append     (from file test_append) -- opens a file in append mode, flushes a specified amount of bytes, waits
#                                           a specified amount of time, and closes the file.
#                                           open, flush, wait, and close operations are repeated until there are bytes to be written.
#
# test_orchestrator imports the three classes and runs one experiment at a time with a specific set of testing parameters:
# - Workload paramters can be specified in the "Workload parameters" section.
# - Wait time between experiments and experiment cycles can be specified in the "Orchestrator wait times" section. 
# - OUTPUT_FOLDER defines the output folder to be used for the experiments.
# - CHECK_DATA enables checksum verification of written files when set to True.
#
# The output produced by test_orchestrator is a plain list of workload parameters and experiments being executed.
# A more refined logging is demanded to testing classes.
#
# PlainWrite, Flush, and Append provide logging capabilities.
# Experiment logs provide the list of operations executed, throughput performance, event timing, 
# and eventual sanity checks. 
# All the three classes will create two subfolders in the specified "OUTPUT_FOLDER":
# - processing: containing produced files
# - logs:       containing experiment logs
#
#--------------------------------------------------------------------------------


import random
import time

#--------------------------------------------------------------------------------
# Import test classes from respective files
#--------------------------------------------------------------------------------
from test_write import PlainWrite
from test_flush import Flush
from test_append import Append


#--------------------------------------------------------------------------------
# Define the output folder
#   If not speficied, the test classes should resort to the current working directory
#--------------------------------------------------------------------------------
# For testing purposes, we impose a strict nomenclature on the folder tree 
# to distinguish between different machines
#
# This is the expected folder tree
# .
# |__ ENTF
# |   |__ $hostname
# |       |__ workdir
# |       |   |  logs
# |       |   |  processing
# |       |__ entf               # Clone of the git repository
# |           |  README.md
# |           |  LICENSE
# |           |__ IO_testing
OUTPUT_FOLDER = "../../workdir"  # This notebook is inside the IO_testing folder on the git repo


#--------------------------------------------------------------------------------
# Workload parameters
#   Define a range of workload paramters with which you want to run the tests
#--------------------------------------------------------------------------------
SIZES = [100000, 200000, 500000, 1000000, 5000000]
TYPES = ["text", "binary"]
INTER_FILE = [0, 0.1, 0.2, 0.5, 1]
CHUNK_SIZE = [10000, 20000, 50000, 100000, 500000]
INTER_CHUNK = [0, 0.01, 0.02, 0.05, 0.1]


#--------------------------------------------------------------------------------
# Choose whether you want the sanity check on written files
#--------------------------------------------------------------------------------
CHECK_DATA = True


#--------------------------------------------------------------------------------
# Choose whether you want to publish the results on Grafana
#--------------------------------------------------------------------------------
GRAFANA = True


#--------------------------------------------------------------------------------
# Orchestrator wait times
#   Define wait times between consecutive tests and test runs
#--------------------------------------------------------------------------------
WAIT_TEST = 60
WAIT_CYCLE = 600


#--------------------------------------------------------------------------------
# Print a legend line for metadata
#--------------------------------------------------------------------------------
print " ".join(["file_number", "file_size", "file_type", "inter_file_time", "chunk_size", "inter_chunk_time"])


#--------------------------------------------------------------------------------
# Run the tests forever
#--------------------------------------------------------------------------------
i=0
while (True):
    i += 1
    print "Test no. %d\n" % i

    #--------------------------------------------------------------------------------
    # Extract randomly a set of workload parameters for the current test
    #--------------------------------------------------------------------------------
    fno = random.choice(range(10))+1          # File number
    fsi = random.choice(SIZES)                # File size
    fty = random.choice(TYPES)                # File types
    ifi = random.choice(INTER_FILE)           # Wait time between write of two files
    csi = random.choice(CHUNK_SIZE)           # Chunk size (for flush and append tests)
    ich = random.choice(INTER_CHUNK)          # Wait time between write of two chunks (for flush and append tests)
    print fno, fsi, fty, ifi, csi, ich

    # Do the tests
    print "\nPlainWrite"
    #def __init__(self, fno, fsize, ftype, itime=0, sanity_check=True, output_folder=CWD, machine_id=None, to_grafana=True):
    current_test = PlainWrite(fno, fsi, fty, ifi, sanity_check=CHECK_DATA, output_folder=OUTPUT_FOLDER, to_grafana=GRAFANA)
    current_test.run()
    time.sleep(WAIT_TEST)

    print "\nFlush"
    #def __init__(self, fno, fsize, ftype, flush_size=4096, flush_time=0, itime=0, sanity_check=False, output_folder=CWD, machine_id=None, to_grafana=True):
    current_test = Flush(fno, fsi, fty, csi, ich, ifi, sanity_check=CHECK_DATA, output_folder=OUTPUT_FOLDER, to_grafana=GRAFANA)
    current_test.run()
    time.sleep(WAIT_TEST)

    print "\nAppend"
    #def __init__(self, fno, fsize, ftype, chunk_size=4096, chunk_time=0, itime=0, sanity_check=False, output_folder=CWD, machine_id=None, to_grafana=True):
    current_test = Append(fno, fsi, fty, csi, ich, ifi, sanity_check=CHECK_DATA, output_folder=OUTPUT_FOLDER, to_grafana=GRAFANA)
    current_test.run()
    time.sleep(WAIT_TEST)

    time.sleep(WAIT_CYCLE)
